In [1]:
!pip3 install -r requirements.txt
!pip3 install flask

Defaulting to user installation because normal site-packages is not writeable
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.2 requires click>=8.0, but you have click 7.1.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit

In [2]:


import cv2
from glob import glob
from pathlib import Path

from model import Model
from utils import plot

/var/data/python/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
model = Model()

# Read video 
### and save predictions for each frame as another video

In [4]:
def process_stream(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    prediction = model.predict(frame)
    label = prediction['label']
    conf = prediction['confidence']
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return label, conf, frame

In [5]:
from flask import Flask, render_template, Response, current_app, redirect, url_for, request
import flask
import socket
import cv2, time


        
app = Flask('hello')


app_context = app.app_context()


notifications = {"0":"Unknown"}

def gen_frames_camera(camera_num):  
    camera = cv2.VideoCapture(camera_num)  # CAP_DSHOW because of https://answers.opencv.org/question/234933/opencv-440modulesvideoiosrccap_msmfcpp-682-cvcapture_msmfinitstream-failed-to-set-mediatype-stream-0-640x480-30-mfvideoformat_rgb24unsupported-media/
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            label, conf, frame = process_stream(frame)
            print(label, conf)
            if (label == "fight on a street") | (label == "street violence") | (label == "violence in office"):
                notifications["0"] = label
                print("hello", label)
                app.get("notification")
                


            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


def gen_frames_drone():
    tello_video = cv2.VideoCapture("udp://@0.0.0.0:11111")
    while True:
        try:
            ret, frame = tello_video.read()

            if ret:
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

        except Exception as err:
            print(err)
            tello_video.release()
        
            # sock.sendto("ccw 22.5".encode(), tello_address)
            # response = sock.recv(1024).decode()
            # print("response #4", response)
            # time.sleep(0.5)



@app.route('/video_feed_0')
def video_feed_0():
    return Response(gen_frames_camera(0), mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/video_feed_1')
def video_feed_1():
    return Response(gen_frames_camera(2), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_2')
def video_feed_2():
    return Response(gen_frames_camera(6), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_drone')
def video_feed_drone():
    return Response(gen_frames_drone(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return flask.render_template('./index.html')

@app.route('/notification')
def notification():
    return flask.render_template('./notification.html', data=notifications)


app.run()

 * Serving Flask app 'hello'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/css/index.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/js/index.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/Ellipse_1.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/Ellipse_12.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/Ellipse_15.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/Vector_20.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/v20_1259.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/v20_1234.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/1.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /static/images/6.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [18/Sep/2022 08:14:05] "GET /st

person walking in office 0.2336273044347763
person walking in office 0.23648299276828766
person walking in office 0.2338528335094452
person walking in office 0.23759354650974274
person walking in office 0.23497642576694489
person walking in office 0.2449847161769867
Unknown 0.22109583020210266
Unknown 0.2185867726802826
Unknown 0.2224772423505783
Unknown 0.21537438035011292
Unknown 0.21773594617843628
Unknown 0.21718913316726685
Unknown 0.22287490963935852
Unknown 0.22292004525661469
Unknown 0.22644244134426117
Unknown 0.22685573995113373
Unknown 0.22788681089878082
Unknown 0.2271425575017929
Unknown 0.22962726652622223
Unknown 0.21584098041057587
Unknown 0.2231508493423462
Unknown 0.21553052961826324
Unknown 0.21132870018482208
Unknown 0.21541115641593933
Unknown 0.2214982509613037
Unknown 0.2163657248020172
Unknown 0.21555688977241516
Unknown 0.2207614630460739
Unknown 0.21898773312568665
Unknown 0.223738893866539
Unknown 0.21293598413467407
Unknown 0.21628807485103607
Unknown 0.2121

Error on request:
Traceback (most recent call last):
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/serving.py", line 335, in run_wsgi
    execute(self.server.app)
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/serving.py", line 324, in execute
    for data in application_iter:
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/wsgi.py", line 500, in __next__
    return self._next()
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/wrappers/response.py", line 50, in _iter_encoded
    for item in iterable:
  File "/tmp/ipykernel_1599/4100524622.py", line 28, in gen_frames_camera
    app.get("notification")
  File "/var/data/python/lib/python3.9/site-packages/flask/scaffold.py", line 49, in wrapper_func
    self._check_setup_finished(f_name)
  File "/var/data/python/lib/python3.9/site-packages/flask/app.py", line 722, in _check_setup_finished
    raise AssertionError(
AssertionError: The setup method 'get' can no longer be called on th

person walking in office 0.23191556334495544
violence in office 0.2336023449897766
hello violence in office


In [ ]:
# import the opencv library
import cv2
  
  
# define a video capture object
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    label, conf, frame = process_stream(frame)
    
    # Display the resulting frame
    cv2.imshow("feed", frame)
    print(label, conf)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()